# Problema de optimización usando Algoritmos Genéticos

## Función Himmelblau

* Función: $f(x,y) = (x^2 +y -11)^2 + (x+y^2-7)^2$
* NO Var: 2
* Precisión: 6
* Espacio de búsqueda : $-5<x,y<5$

# Parámetros del algoritmo

* Método de selección: ruleta
* Cruza: En un punto
* Mutación: en un bit
* Número de generación: 30
* Tamaño de la población (número de individuos): 50
* Probabilidad de cruza: $P_c = 0.85$
* Probabilidad de mutación : 0.000000001

## Generación de la población

### Codificación de las variables

$$
L = [\log_2((l_{sup} - l_{inf})\times 10^{\text{precision}})]
$$


In [11]:
from math import log2, ceil


def length_variable(l_sup: int, l_inf: int , precision: int):
    """
    Nombre de la función : length_variable
    
    Descripción: Función que obtiene la cantidad de bits para representar la variable
    
    argumentos:
        * l_sup: límite superior de la función
    """
    return ceil(log2((l_sup - l_inf)*10**precision))

x_l_sup = 5
y_l_sup = 5

x_l_inf = -5
y_l_inf = -5

precision = 6

L_x = length_variable(x_l_sup, x_l_inf, precision)
L_y = length_variable(y_l_sup, y_l_inf, precision)
L_x, L_y

(24, 24)

In [24]:
# Función que genera poblacipob aleatori de m individuos de tamaño L
import random

m = 30 

def rand_population_binary(m,L):
    return [[random.randint(0,1)  for i in range(L)] for j in range(m)]

rand_population_binary(m,L_x)[0]

[0, 0, 1, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0]

In [25]:
rand_population_binary(m,L_y)[0]

[0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0]

In [36]:
# GENERAMOS LA POBLACIÓN INICIAL

import numpy as np

n_variables = 2
l_sup_vec = [x_l_sup, y_l_sup]
l_inf_vec = [x_l_inf, y_l_inf]

dimension = []
genotipo = []
L_cromosoma = 0

# Generamos la poblacion inicial

for i in range(n_variables):
    L_var = length_variable(l_sup_vec[i], l_inf_vec[i], precision)
    L_cromosoma +=  L_var
    
    dimension.append(L_var)
    genotipo.append(rand_population_binary(m, L_var))

genotipo

[[[0, 0, 1, 1, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 0, 0, 1, 0],
  [1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 1],
  [0, 1, 0, 1, 0, 1, 1, 0, 0, 0, 1, 1, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 1, 0],
  [1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 1, 0, 0, 0],
  [1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 1, 1],
  [1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1],
  [0, 1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1],
  [1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0],
  [0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 1],
  [1, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 1, 1, 1, 1, 0, 1, 0, 0, 0, 1, 1, 1],
  [1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 1],
  [1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1],
  [1, 0, 0, 0, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 1, 1, 1, 1, 0, 1],
  [1, 0, 0, 

# Evaluación de población inicial

In [55]:
# Función que obtiene las potencias base 2 de un vector de bits (un individui)

def to_decimal(dimension,individuo):
    return sum([2**(i) for i in range(dimension-1,-1,-1) ]* np.array(individuo))

In [57]:
dimension = 24
individuo_inicial = genotipo[0][4]

to_decimal(dimension, individuo_inicial)

10261647

In [66]:
# Función que decodifica el vector a un valor real

def binary2real(i_sup, i_inf, dimension, individuo):
    return i_inf+ (to_decimal(dimension, individuo)* ((i_sup-i_inf)/(2**len(individuo)-1)))

In [69]:
for i in genotipo[0]:
    print(binary2real(5, -5, 24, i))

-3.0666528979929026
0.261699870926134
-1.632445849922052
3.9665561298463423
1.1164186070214868
1.4275763289675911
-0.12760312125701478
4.826672662894289
-1.7799667584876273
3.058972243009343
2.52100214487327
2.649340489467412
0.5427918161625751
0.38588138734587396
-1.971221385670983
2.0854352167508132
-1.6953633246042328
2.7124022073985463
3.1744544610055954
-0.5064538423093463
-2.2706340116640336
-3.0369709752184733
0.30177505622953493
2.6930759962246418
-3.3010058582428607
4.436794485854774
-4.5805668580869945
2.1586178039680606
2.6691739361985887
4.6603357589444965
